In [1]:
import pandas as pd
import pickle as pkl
import openai
from sklearn.metrics import confusion_matrix
import time
import json

# import large file
with open('../ct_df_posts_single_preds.pkl', 'rb') as f:
    pre_df = pkl.load(f)

# shorten to not murder my computer
# df = pre_df.iloc[:50]
#del pre_df

# retrieve API key
import os
api_key = os.environ.get('OPENAI_API_KEY')
# print(api_key)

In [2]:
# only get the ones without disclosures
pre_df = pre_df[pre_df["has_disclosures"]==False]

# sample 50 ads, 50 non-ads
df = pd.concat([pre_df[pre_df.loc[:,"predicted_disclosure"]==True].sample(50),
                pre_df[pre_df.loc[:,"predicted_disclosure"]==False].sample(50)]).copy()
df.reset_index(drop=True,inplace=True)
del pre_df

In [7]:
# Generate classifications for all samples

completions = {"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}
for model in ["text-davinci-002","text-ada-001", "text-babbage-001", "text-curie-001"]:
    i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i==24: time.sleep(65)
        if i==49: time.sleep(65)
        if i==74: time.sleep(65)
        if i==99: time.sleep(65)

        prompt1 = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'False'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"
        
        prompt2 = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'True'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"

        # Generate a response from openai.
        # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
        prompt=("warning")
        if model in ["text-ada-001", "text-babbage-001"]:
            if i==1: print("pompt1")
            prompt=prompt1
        elif model in ["text-curie-001", "text-davinci-002"]:
            if i==1: print("pompt2")
            prompt=prompt2
        else:
            print("warning")
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=5,
            n=3,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[model].append(response["choices"][0]["text"])
    print(model+" done")

pompt2
Counter at 10
Counter at 20
Counter at 30
Counter at 40
Counter at 50
Counter at 60
Counter at 70
Counter at 80
Counter at 90
Counter at 100
text-davinci-002 done
pompt1
Counter at 10
Counter at 20
Counter at 30
Counter at 40
Counter at 50
Counter at 60
Counter at 70
Counter at 80
Counter at 90
Counter at 100
text-ada-001 done
pompt1
Counter at 10
Counter at 20
Counter at 30
Counter at 40
Counter at 50
Counter at 60
Counter at 70
Counter at 80
Counter at 90
Counter at 100
text-babbage-001 done
pompt2
Counter at 10
Counter at 20
Counter at 30
Counter at 40
Counter at 50
Counter at 60
Counter at 70
Counter at 80
Counter at 90
Counter at 100
text-curie-001 done


In [4]:
# save interrupted run
temp_completions = completions

In [8]:
# Make predictions to booleans
completions_as_boolean = {}
for model in ["text-ada-001", "text-babbage-001", "text-curie-001", "text-davinci-002"]:
    completions_as_boolean[model] = [True if response.__contains__("rue") 
                          else False if response.__contains__("als")
                          else "warning" for response in completions[model]]

    # check for bad completions
    if "warning" in completions[model]: print("Warning")

    # obtain confusion matrix

    print(confusion_matrix(df["predicted_disclosure"],completions_as_boolean[model]))

[[ 7 43]
 [ 2 48]]
[[36 14]
 [32 18]]
[[45  5]
 [31 19]]
[[49  1]
 [43  7]]


In [10]:
completions["text-curie-001"]

[' True',
 ' False',
 ' True',
 ' False',
 ' True',
 ' False',
 ' False',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False',
 ' True',
 ' True',
 ' False',
 ' False',
 ' False',
 ' True',
 ' False',
 ' True',
 ' True',
 ' False',
 ' True',
 ' False',
 ' True',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' True',
 ' True',
 ' True',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 '\n\nTrue',
 ' True',
 ' False',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False',
 ' True',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' True',
 ' False',
 '\n\nFalse',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 '\n\nTrue',
 ' False',
 ' False',
 ' False',
 ' False',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' False',
 ' 

In [ ]:
with open('.pkl', 'rb') as f:
    pre_df = pkl.load(f)

In [13]:
# next steps: translate strings to bool and compare different results to what he got

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [19]:
# before doing this, add something like the true values
with open('results.pkl', 'wb') as file:
    pkl.dump(completions, file)

### Varying the prompt to get davinci to more positive judgements

In [19]:
# Generate classifications for all samples

completions ={} #{"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}

for x in range(2):#,"text-ada-001", "text-babbage-001", "text-curie-001"]:
    i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i==24: time.sleep(65)
        if i==49: time.sleep(65)
        if i==74: time.sleep(65)
        if i==99: time.sleep(65)
        
        if x == 0:
            prompt = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are very uncertain err rather towards judging 'True'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"
        elif x ==1:
            prompt = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. If you are not sure, judge 'True'.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"

        

        # Generate a response from openai.
        # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
        #prompt=("warning")
        #if model in ["text-ada-001", "text-babbage-001"]:
        #    if i==1: print("pompt1")
        #    prompt=prompt1
        #elif model in ["text-curie-001", "text-davinci-002"]:
        #    if i==1: print("pompt2")
        #    prompt=prompt2
        #else:
        #    print("warning")
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=5,
            n=3,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[x].append(response["choices"][0]["text"])
    print(model+" done")

KeyError: 0